In [2]:
import pandas as pd
import datetime
from utils.clean_functions import *


## LIMPIEZA

### df_refugees (acumulado de refugiados ucranianos por fecha y destino)

- Importo el dataframe i veo que no tiene nulos

In [3]:
df_refugees = pd.read_csv(r'.\data\Refugees.csv')

df_refugees.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    665 non-null    int64  
 1   id            665 non-null    object 
 2   country       665 non-null    object 
 3   date          665 non-null    object 
 4   individuals   665 non-null    int64  
 5   centroid_lon  665 non-null    float64
 6   centroid_lat  665 non-null    float64
 7   lat_max       665 non-null    float64
 8   lon_max       665 non-null    float64
 9   lan_min       665 non-null    float64
 10  lon_min       665 non-null    float64
 11  source        665 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 62.5+ KB


- Creo una función que convierte la columna de fechas ("date") en un datetime ordenado (index_by_datetime) y lo lleva al índice, lo cual me será útil si tengo que unificar tablas
- Dejo solo las columnas de país ("country") y acumulado de refugiados ("individuals")


In [4]:
index_by_datetime(df_refugees)

df_refugees = df_refugees[['country','individuals']]


- Compruebo que hay varios países que aparecen bastante menos que los demás. Sin embargo, como todavía no he igualado esta tabla a la otra necesaria para la hipótesis (es decir, la de precios), no tengo claro qué meses voy a necesitar ni qué días de esos meses serán el mejor punto de referencia (por aparecer en la otra tabla). Por tanto, de momento, los dejo todos

In [5]:
df_refugees['country'].value_counts()


Hungary                     106
Slovakia                    105
Poland                      102
Republic of Moldova         102
Romania                      93
Belarus                      76
Russian Federation           70
Other European countries     11
Name: country, dtype: int64

- Resultado:

In [6]:
df_refugees.head()


,country,individuals
date,,
2022-03-01,Belarus,341
2022-03-01,Poland,453982
2022-03-01,Hungary,116348
2022-03-01,Republic of Moldova,79315
2022-03-01,Russian Federation,42900


### df_prices (precios en cada mercado de Ucrania por fecha, tipo y longitud Este)

- Importo el dataframe y veo que hay algunos nulos

In [7]:
df_prices = pd.read_csv(r'.\data\Prices.csv', low_memory=False)

df_prices.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70731 entries, 0 to 70730
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       70731 non-null  object
 1   admin1     69685 non-null  object
 2   admin2     69685 non-null  object
 3   market     70731 non-null  object
 4   latitude   69685 non-null  object
 5   longitude  69685 non-null  object
 6   category   70731 non-null  object
 7   commodity  70731 non-null  object
 8   unit       70731 non-null  object
 9   priceflag  70731 non-null  object
 10  pricetype  70731 non-null  object
 11  currency   70731 non-null  object
 12  price      70731 non-null  object
 13  usdprice   70731 non-null  object
dtypes: object(14)
memory usage: 7.6+ MB


- Al mirar la cabecera de la tabla, y teniendo en cuenta que hay el mismo número de nulos en "admin1", "admin2", "latitude" y "longitude", salta a la vista que probablemente correspondan todos al valor de "market" llamado "National Average". Esa media no me interesa para el análisis

In [8]:
df_prices.head()


,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2014-03-15,NaN,NaN,National Average,NaN,NaN,cereals and tubers,Bread (rye),Loaf,actual,Retail,UAH,4.96,0.135
2,2014-03-15,NaN,NaN,National Average,NaN,NaN,cereals and tubers,Bread (wheat),Loaf,actual,Retail,UAH,4.84,0.1318
3,2014-03-15,NaN,NaN,National Average,NaN,NaN,cereals and tubers,Buckwheat grits,KG,actual,Retail,UAH,7.44,0.2026
4,2014-03-15,NaN,NaN,National Average,NaN,NaN,cereals and tubers,Potatoes,KG,actual,Retail,UAH,6.74,0.1835


- Elimino el índice 0, puramente explicativo
- Elimino todas las filas donde el mercado sea, en realidad, la media nacional ("National Average")
- Cambio a numéricas las columnas "longitude" y "usdprice"
- Aplico la función que me pone la fecha en el índice (index_by_datetime)

In [10]:
df_prices = df_prices.drop(index=0)
df_prices = df_prices[df_prices['market'] != 'National Average']

df_prices[['usdprice', 'longitude']] = df_prices[['usdprice', 'longitude']].apply(pd.to_numeric, errors='coerce', axis=1)

index_by_datetime(df_prices)


- Limito el dataframe al período de la guerra con Russia

In [12]:
start_war = datetime.datetime(2022, 2, 24, 0, 0, 0)
df_prices = df_prices[df_prices.index >= start_war]


- Reduzco el dataframe a las columnas que me interesan para la hipótesis

In [13]:
df_prices = df_prices[['usdprice', 'market', 'commodity', 'longitude']]


Ahora debo elegir qué productos (en "commodity") me servirán para comparar las oscilaciones de precios en función de las bajas, los refugiados o la lejanía al frente. A ver qué opciones hay

In [14]:
df_prices.commodity.value_counts()


Wheat flour                          156
Rice                                 155
Buckwheat                            151
Sugar                                150
Antipyretic (local)                  150
Potatoes                             150
Fuel (petrol-gasoline, 92 octane)    149
Meat (beef)                          149
Vasodilating agents (imported)       148
Antipyretic (imported)               148
Millet                               148
Oil (sunflower)                      147
Carrots                              147
Onions                               147
Meat (chicken, fillet)               147
Fuel (diesel)                        146
Curd                                 145
Fat (salo)                           145
Fuel (petrol-gasoline, 95 octane)    145
Meat (pork)                          145
Semolina                             144
Pasta                                144
Cabbage                              144
Milk                                 144
Barley          

- Noto que los medicamentos vienen en 3 tipos (antibióticos, antipiréticos y agentes vasodilatadores), y que los tres pueden ser locales o importados, lo cual me ofrece dos dimensiones más para comparar. Además, como en la guerra suele haber más heridos que en períodos de paz, será más probable que se vean afectados en caso de encontrarse un mercado en zona de conflicto. Así pues, elijo conservar los medicamentos para mi tabla

In [15]:
df_prices = df_prices[(df_prices['commodity'].str.contains('imported')) |(df_prices['commodity'].str.contains('local'))]


- Resultado:

In [16]:
df_prices.head()


,usdprice,market,commodity,longitude
date,,,,
2022-03-15,4.8148,Rivne,Antibiotics (imported),26.251617
2022-03-15,1.0399,Rivne,Antibiotics (local),26.251617
2022-03-15,0.4625,Rivne,Antipyretic (local),26.251617
2022-03-15,0.3667,Rivne,Vasodilating agents (local),26.251617
2022-03-15,1.5907,Rivne,Vasodilating agents (imported),26.251617


### df_personnel (pérdidas rusas personales por fecha)

- Importo el dataframe y veo que hay algunos nulos

In [17]:
df_personnel = pd.read_csv(r'.\data\Russia_losses_personnel(date).csv')

df_personnel.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        258 non-null    object 
 1   day         258 non-null    int64  
 2   personnel   258 non-null    int64  
 3   personnel*  258 non-null    object 
 4   POW         62 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 10.2+ KB


La columna de prisioneros de guerra ("POW") no me es útil para el análisis, y las de "personnel*" y "day", tampoco. Las elimino

In [18]:
df_personnel = df_personnel[['date', 'personnel']]


- Creo una función que me permite convertir columnas de valores acumulados en valores absolutos (decumulate_columns), y se la aplico a "personnel"
- Utilizo, una vez más, la función que pasa la fecha al índice en forma de datetime (index_by_datetime)

In [19]:
decumulate_columns(df_personnel, excluded=['date'])
index_by_datetime(df_personnel)


- Resultado:

In [20]:
df_personnel.head()


,personnel
date,
2022-02-25,2800
2022-02-26,1500
2022-02-27,200
2022-02-28,800
2022-03-01,410


### df_equipment (pérdidas rusas materiales, según categorías generales, por fecha)

- Importo el dataframe y veo que hay nulos en demasía

In [21]:
df_equipment = pd.read_csv(r'.\data\Russia_losses_equipment(date).csv')

df_equipment.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       258 non-null    object 
 1   day                        258 non-null    int64  
 2   aircraft                   258 non-null    int64  
 3   helicopter                 258 non-null    int64  
 4   tank                       258 non-null    int64  
 5   APC                        258 non-null    int64  
 6   field artillery            258 non-null    int64  
 7   MRL                        258 non-null    int64  
 8   military auto              65 non-null     float64
 9   fuel tank                  65 non-null     float64
 10  drone                      258 non-null    int64  
 11  naval ship                 258 non-null    int64  
 12  anti-aircraft warfare      258 non-null    int64  
 13  special equipment          239 non-null    float64

Vamos a ver como se distribuyen los nulos en porcentaje

In [22]:
((df_equipment.isnull().sum() / len(df_equipment))*100).sort_values(ascending = False)


mobile SRBM system           86.046512
fuel tank                    74.806202
military auto                74.806202
greatest losses direction    29.457364
vehicles and fuel tanks      25.193798
cruise missiles              25.193798
special equipment             7.364341
field artillery               0.000000
MRL                           0.000000
APC                           0.000000
day                           0.000000
drone                         0.000000
naval ship                    0.000000
anti-aircraft warfare         0.000000
tank                          0.000000
helicopter                    0.000000
aircraft                      0.000000
date                          0.000000
dtype: float64

Una pequeña investigación me permite clasificar qué hay exactamente en las columnas de nulos. A parte de las columnas "greatest losses direction" y "day", que no las necesitamos porque no son sobre equipamiento, vemos que:
- Las columnas "military auto", "fuel tank" y "mobile SRBM system" están demasiado incompletas como para que tenga sentido usarlas, así que no hace falta darles más vueltas. Se eliminan
- "Vehicles and fuel tanks" puede no considerarse material de guerra al uso (fuel tanks no son tanques como se entendería en este contexto, sino meros silos de combustible), así que la quito
- "Cruise missile" es misil guiado en español. Es razonable considerarlo un tipo de munición pesada más que una arma en sí. Además, será destruido tanto si tiene éxito en su cometido como si no. Su comportamiento anómalo solo servirá para desvirtuar la tabla incluso tras una interpolación de los valores que contiene. Borro la columna
- Solo queda "special equiment". Podríamos rellenar los nulos de esta columna con la media/mediana y salvarla, ya que no son tantos, pero es mejor deshacerse de ella. La razón es que ni el propio creador del dataset en kaggle tiene claro qué contiene (si bien intuye que se trata de cosas como radios o misiles: https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war/discussion/316981). Si la teoría del autor es correcta, y contiene misiles, es similar a la columna "Cruise missile" que hemos visto en el punto anterior, por lo que conviene darle el mismo trato

In [23]:
df_equipment.dropna(axis=1, inplace=True)
df_equipment.drop(columns=['day'], inplace=True)


- Desacumulo las columnas (decumulate_columns) y llevo la fecha al índice (index_by_datetime)
- Añado una columna con la suma de los equipamientos


In [24]:
decumulate_columns(df_equipment, excluded=['date'])
index_by_datetime(df_equipment)

df_equipment['total losses'] = df_equipment[['aircraft', 
                                        'helicopter',
                                        'tank',
                                        'APC',
                                        'field artillery',
                                        'MRL',
                                        'drone',
                                        'naval ship',
                                        'anti-aircraft warfare'
                                        ]].sum(axis=1)


- Resultado:

In [25]:
df_equipment.head()

,aircraft,helicopter,tank,APC,field artillery,MRL,drone,naval ship,anti-aircraft warfare,total losses
date,,,,,,,,,,
2022-02-25,10,7,80,516,49,4,0,2,0,668
2022-02-26,17,19,66,190,0,0,2,0,0,294
2022-02-27,0,0,4,0,1,0,0,0,0,5
2022-02-28,2,3,0,110,24,17,1,0,5,162
2022-03-01,0,0,48,30,3,3,0,0,2,86


### df_tech (pérdidas materiales rusas, incluyendo modelo y fabricante, sin fecha)

- Importo el dataframe y veo que hay nulos en cantidades industriales

In [26]:
df_tech = pd.read_csv(r'.\data\Russia_losses_equipment(tech_details).csv')

df_tech.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   equipment                                    298 non-null    object 
 1   model                                        298 non-null    object 
 2   sub_model                                    8 non-null      object 
 3   manufacturer                                 298 non-null    object 
 4   losses_total                                 298 non-null    int64  
 5   abandoned                                    77 non-null     float64
 6   abandoned and destroyed                      17 non-null     float64
 7   captured                                     196 non-null    float64
 8   captured and destroyed                       7 non-null      float64
 9   captured and stripped                        4 non-null      float64
 10  da

Compruebo cuántos nulos hay en total

In [27]:
df_tech.isnull().sum().sum()


3740

- Son muchos. Ahora bien, ¿cuántos de esos nulos son, en realidad, ceros (porque ha habido 0 bajas de ese tipo para tal pieza de equipamiento)? La manera de saberlo es fijarse en la columna "losses_total"; el total de bajas para cada pieza debería ser una suma de las destruidas y las capturadas. Si se queda corto, es que faltan datos
- Empiezo por cambiar los NaN por 0
- Sumo los capturados y los pongo en una nueva columna, y hago lo propio con el resto de bajas
- Limito el dataframe a las columnas que he creado y a las especificaciones del equipamiento (excepto "sub_model", que es innecesaria para la hipótesis)

In [28]:
df_tech.fillna(0.0, inplace=True)

df_tech['total captured'] = df_tech[['captured', 
                                    'captured and destroyed',
                                    'captured and stripped',
                                    'damaged and captured'
                                    ]].sum(axis=1)

df_tech['total not captured'] = df_tech[['abandoned', 
                                        'abandoned and destroyed',
                                        'damaged',
                                        'damaged and abandoned',
                                        'damaged beyond economical repair',
                                        'damaged by Bayraktar TB2',
                                        'destroyed',
                                        'destroyed by Bayraktar TB2',
                                        'destroyed by Bayraktar TB2 and Harpoon AShM',
                                        'sunk'
                                        ]].sum(axis=1)

df_tech = df_tech[['equipment','model', 'manufacturer', 'losses_total', 'total captured', 'total not captured']]


- Hago la comprobación. Parece que el total de pérdidas coincide con la suma de las partes para cada pieza de equipo. Por ende, no faltan datos a pesar de que hay multitud de nulos

In [29]:
all(df_tech['losses_total']) == all(df_tech['total captured'] + df_tech['total not captured'])


True

- Resultado

In [30]:
df_tech.head()


,equipment,model,manufacturer,losses_total,total captured,total not captured
0,Tanks,T-62M,the Soviet Union,20,16.0,4.0
1,Tanks,T-62MV,the Soviet Union,3,2.0,1.0
2,Tanks,T-64A,the Soviet Union,2,0.0,2.0
3,Tanks,T-64BV,the Soviet Union,39,5.0,34.0
4,Tanks,T-72A,the Soviet Union,33,15.0,18.0


### df_uk_tech (pérdidas materiales ucranianas, incluyendo modelo y fabricante, sin fecha)

- Este dataframe es idéntico al anterior, pero con pérdidas ucranianas. Por tanto, el tratamiento es el mismo

In [31]:
df_uk_tech = pd.read_csv(r'.\data\Ukraine_losses_equipment(tech_details).csv')

df_uk_tech['total captured'] = df_uk_tech[['captured', 
                                            'captured and destroyed',
                                            'damaged and captured',
                                            'damaged by Orion and captured',
                                            'sunk but raised by Russia'
                                            ]].sum(axis=1)

df_uk_tech['total not captured'] = df_uk_tech[['abandoned', 
                                                'abandoned and destroyed',
                                                'damaged',
                                                'damaged and abandoned',
                                                'damaged beyond economical repair',
                                                'damaged by Forpost-R',
                                                'destroyed',
                                                'destroyed by Forpost-R',
                                                'destroyed by Orion',
                                                'destroyed by loitering munition',
                                                'scuttled to prevent capture by Russia',
                                                'sunk'
                                                ]].sum(axis=1)

df_uk_tech = df_uk_tech[['equipment','model', 'manufacturer', 'losses_total', 'total captured', 'total not captured']]


Por si las moscas, hago la comprobación

In [32]:
all(df_uk_tech['losses_total']) == all(df_uk_tech['total captured'] + df_uk_tech['total not captured'])


True

- Resultado

In [33]:
df_uk_tech.head()


,equipment,model,manufacturer,losses_total,total captured,total not captured
0,Tanks,T-64A,the Soviet Union,1,1.0,0.0
1,Tanks,T-64B,the Soviet Union,1,0.0,1.0
2,Tanks,T-64BV,the Soviet Union,123,53.0,70.0
3,Tanks,T-64BV Zr. 2017,Ukraine,49,27.0,22.0
4,Tanks,T-64B1M,Ukraine,4,4.0,0.0
